In [1]:
import numpy as np
from scipy.stats import chisquare
from scipy.stats import gmean
import cvxopt as opt
from cvxopt import matrix, spmatrix, sparse
from cvxopt.solvers import qp, options
from cvxopt import blas
import pandas as pd

C:\Users\samga\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def geo_mean(returns):
    geo = []
    n = len(returns.transpose())
    for i in range(0,n):
        geo = geo + [np.exp(np.log(rets[:,i]+1).mean())-1]
    return geo

def quad_opt_func(Q,n):
    
    return 2*Q


def lin_opt_func(r,n):
    if r == False:
        return np.zeros([n,1])
    else:
        return r
    
def inequality_constraints(n):
    
    # Inequality Constraint
    # Expected Return Over 0.0035
    G1 = np.matrix([-1])*mu
    h1 = np.matrix(-0.00035)

    # Lower Bound on Each Element!
    G2 = -1*np.identity(n)
    h2 = np.zeros([n,1])

    #Concat all Answers
    G = G2#np.concatenate((G1,G2),axis=0)
    h = h2#np.concatenate((h1,h2),axis=0)
    
    return G,h

def equality_constraints(n):
    
    # Equality Constraint
    # Weight sum is 1
    A1 = np.ones([1,n])
    b1 = np.ones([1,1])
    
    
    #Concat All Equality
    A = A1#np.concatenate((A1),axis=0)
    b = b1#np.concatenate((b1),axis=0)
    
    return A,b

def MVO(mu,Q,x0):
    
    #NOTE: X0 is not used yet but it will be for transaction costs
    
    # Number of Assets
    n = len(Q)
    
    # ----- Constraints -----------
    
    # Equality Constraint
    
    A,b = equality_constraints(n)
    
    # Inequality Constraint
    
    G,h = inequality_constraints(n)
    
    
    # --- Quadtratic Optimization Function --------
    #quad = 2*Q
    quad = quad_opt_func(Q,n)
    # ------ Linear Optimization Function ---------
    #r = np.zeros([n,1])
    r = lin_opt_func(mu,n)
    
    
    # ------------ Optimize! --------
    sol = qp(matrix(quad), -1*matrix(r), matrix(G), matrix(h), matrix(A), matrix(b))['x']
    #sol = qp(matrix(quad), matrix(r), None, None, matrix(A), matrix(b))#['x']
    
    
    return sol
    
    